# Reducing Hallucinations from AI Agents using Long-Term Memory
### Introduction to Critique-Based Contexting with OpenAI, LangChain, and LanceDB
AI agents can help simplify and automate tedious workflows. By going through this notebook, we'll introduce how you can reduce hallucinations from AI agents by using critique-based contexting with a fitness trainer agent example.

### Installing dependencies

In [2]:
!pip install openai==0.28 langchain==0.0.354 google-search-results lancedb python-dotenv -q

### Importing libraries

In [3]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

Now let's import our environment variables via `load_dotenv()`.

In [4]:
from dotenv import load_dotenv

load_dotenv()

False

We now specify and connect to the path `data/agent-lancedb` to store our vector database.

In [5]:
import lancedb

db = lancedb.connect("data/agent-lancedb")

To create embeddings out of the text, we'll call the OpenAI embeddings API (ada2 text embeddings model) to get embeddings.

In [13]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "sk-proj-...."  # PASTE YOUR KEY HERE


def embed_func(c):
    rs = openai.Embedding.create(input=c, engine="text-embedding-ada-002")
    return [record["embedding"] for record in rs["data"]]

Now, we'll create a `LangChain` tool that allows our agent to insert critiques, which uses a `pydantic` schema to guide the agent on what kind of results to insert.

In LanceDB the primary abstraction you'll use to work with your data is a **Table**.  
A Table is designed to store large numbers of columns and huge quantities of data! For those interested, a LanceDB is columnar-based, and uses Lance, an open data format to store data.

This tool will create a Table if it does not exist and store the relevant information (the embedding, actions, and critiques).

### Considering the image below
1. ideation step gets a predefined number of output proposals (ideas) from the LLM.

2. critiques all of the ideas looking for possible flaws in the proposals and picking the most appropriate suggestion.

3. resolve, the LLM tries to improve the best idea from the [2] critique step. The output here constitutes the final answer.

![image](https://miro.medium.com/v2/resize:fit:1400/1*dMfLxIR-s8osxkVUbORtGg.png)

### Function to take input

In [10]:
from langchain.tools import tool
from pydantic import BaseModel, Field


class InsertCritiquesInput(BaseModel):
    info: str = Field(
        description="should be demographics or interests or other information about the exercise request provided by the user"
    )
    actions: str = Field(
        description="numbered list of langchain agent actions taken (searched for, gave this response, etc.)"
    )
    critique: str = Field(
        description="negative constructive feedback on the actions you took, limitations, potential biases, and more"
    )


# @tool("insert_critiques", args_schema=InsertCritiquesInput)
def insert_critiques(info: str, actions: str, critique: str) -> str:
    "Insert actions and critiques for similar exercise requests in the future." ""
    table_name = "exercise-routine"
    if table_name not in db.table_names():
        tbl = db.create_table(
            table_name,
            [{"vector": embed_func(info)[0], "actions": actions, "critique": critique}],
        )
    else:
        tbl = db.open_table(table_name)
        tbl.add(
            [{"vector": embed_func(info)[0], "actions": actions, "critique": critique}]
        )
    return "Inserted and done."

Similarly, let's create a tool for retrieving critiques. We'll retrieve the actions and critiques from the top 5 most similar user inputs.

### Retrieve Input

In [11]:
class RetrieveCritiquesInput(BaseModel):
    query: str = Field(
        description="should be demographics or interests or other information about the exercise request provided by the user"
    )


# @tool("retrieve_critiques", args_schema=RetrieveCritiquesInput)
def retrieve_critiques(query: str) -> str:
    "Retrieve actions and critiques for similar exercise requests." ""
    table_name = "exercise-routine"
    if table_name in db.table_names():
        tbl = db.open_table(table_name)
        results = (
            tbl.search(embed_func(query)[0])
            .limit(5)
            .select(["actions", "critique"])
            .to_pandas()
        )
        results_list = results.drop("vector", axis=1).values.tolist()
        return (
            "Continue with the list with relevant actions and critiques which are in the format [[action, critique], ...]:\n"
            + str(results_list)
        )
    else:
        return "No info, but continue."

Let's now use LangChain to load our tools in. This includes our custom tools as well as a Google Search tool that uses SerpApi. We will use OpenAI's `gpt-3.5-turbo-0613` as our LLM.
Note: You need to sign up on serpapi and paste the API key here

In [19]:
os.environ["SERPAPI_API_KEY"] = "....."

llm = ChatOpenAI(temperature=0, model="gpt-4o")
tools = load_tools(["serpapi"], llm=llm)
tools.extend([insert_critiques, retrieve_critiques])

Before we run our agent, let's create a function that defines our prompt that we pass in to the agent, which allows us to pass in client information.

### Create Prompt Function

In [20]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool


def create_prompt(info: str) -> str:
    prompt_start = (
        "Please execute actions as a fitness trainer based on the information about the user and their interests below.\n\n"
        + "Info from the user:\n\n"
    )
    prompt_end = (
        "\n\n1. Retrieve using user info and review the past actions and critiques if there is any\n"
        + "2. Keep past actions and critiques in mind while researching for an exercise routine with steps which we respond to the user\n"
        + "3. Before returning the response, it is of upmost importance to insert the actions you took (numbered list: searched for, found this, etc.) and critiques (negative feedback: limitations, potential biases, and more) into the database for getting better exercise routines in the future. \n"
    )
    return prompt_start + info + prompt_end

### Run Agent

Finally, let's create our run_agent function. We'll use the `STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` agent in order to allow us to use multi-input tools (since we need to add client input, actions, and critiques as arguments).

In [21]:
def run_agent(info):
    # Change agent= to agent_type=
    agent = initialize_agent(
        tools,
        llm,
        agent_type=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
    )
    # Add a return or print to see the result
    return agent.run(input=create_prompt(info))


# You'll need to define these before running
# Example placeholder definitions
tools = [
    Tool.from_function(
        func=lambda x: "Sample fitness advice",
        name="fitness_tool",
        description="Provides fitness advice based on user information",
    )
]
llm = ChatOpenAI(temperature=0)  # You may need to set up API keys

Let's run. Feel free to use your own input!

Notice that in the first run there wouldn't be any critiques yet, since the database is empty. After the first run, critiques should appear. The provided output is the result of a particular run after a few runs.

In [24]:
# Now try running the agent
result = run_agent(
    "My name is Tevin, I'm a 19 year old university student at CMU. I love running."
)



> Entering new AgentExecutor chain...
I should gather information about Tevin's fitness level, running experience, and any specific goals he may have.
Action: fitness_tool
Action Input: Tevin's age, university, love for running
Observation: Sample fitness advice
Thought:I should now tailor an exercise routine for Tevin based on his love for running and fitness level.
Action: fitness_tool
Action Input: Tevin's running experience, fitness goals
Observation: Sample fitness advice
Thought:I should make sure to include a variety of exercises to keep Tevin engaged and prevent boredom.
Action: fitness_tool
Action Input: Tevin's preferred types of exercises
Observation: Sample fitness advice
Thought:I now know the final answer
Final Answer: I have tailored an exercise routine for Tevin based on his love for running, fitness level, running experience, and fitness goals.

> Finished chain.


In [25]:
print(result)

I have tailored an exercise routine for Tevin based on his love for running, fitness level, running experience, and fitness goals.
